In [1]:
os.environ["AWS_PROFILE"] = "Insight"
from create_ledger import *
from connect_to_ledger import *
from create_table import *
from create_index import * 
from insert_document import *

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [22]:
def index_setup(ledgerName, collectionIndex=None, trainIndex=None, predictIndex=None):
    logger.info('Creating indexes on all tables in a single transaction...')
    try:
        for cIndex in collectionIndex:
            with create_qldb_session(ledgerName) as session:
                session.execute_lambda(lambda x: create_index(x, "Collection",
                                                            cIndex),
                                    lambda retry_attempt: logger.info('Retrying due to OCC conflict...'))
                logger.info('Indexes created successfully.')
        for tIndex in trainIndex:
            with create_qldb_session(ledgerName) as session:
                session.execute_lambda(lambda x: create_index(x, "Training",
                                                            tIndex),
                                    lambda retry_attempt: logger.info('Retrying due to OCC conflict...'))
                logger.info('Indexes created successfully.')
        for pIndex in predictIndex:
            with create_qldb_session(ledgerName) as session:
                session.execute_lambda(lambda x: create_index(x, "Prediction",
                                                            pIndex),
                                    lambda retry_attempt: logger.info('Retrying due to OCC conflict...'))
                logger.info('Indexes created successfully.')
    except Exception:
        logger.exception('Unable to create indexes.')

In [28]:
def qldb_setup(ledgerName, collectionIndex=None, trainIndex=None, predictIndex=None):
    create_ledger(ledgerName)
    wait_for_active(ledgerName)
    try:
        with create_qldb_session(ledgerName) as session:
            session.execute_lambda(lambda x: create_table(x, "Collection") and
                                create_table(x, "Training") and
                                create_table(x, "Prediction"),
                                lambda retry_attempt: logger.info('Retrying due to OCC conflict...'))
            logger.info('Tables created successfully.')
    except Exception:
        logger.exception('Errors creating tables.')
    index_setup(ledgerName, collectionIndex, trainIndex, predictIndex)        

In [29]:
cIndex = ['GitHash', 'ScrapeTime', 'DataPath', 'OutputHash']
tIndex = ['GitHash', 'InputHash', 'TrainTime', 'Model']
pIndex = ['GitHash', 'InputHash', 'Model', 'PredictionTime', 'OutputHash']
qldb_setup("Week2Demo", cIndex, tIndex, pIndex)

INFO:create_ledger:Let's create the ledger named: Week2Demo...
INFO:create_ledger:Success. Ledger state: CREATING.
INFO:create_ledger:Waiting for ledger to become active...
INFO:create_ledger:The ledger is still creating. Please wait...
INFO:create_ledger:The ledger is still creating. Please wait...
INFO:create_ledger:Success. Ledger is active and ready to use.
INFO:create_table:Creating the 'Collection' table...
INFO:create_table:Collection table created successfully.
INFO:create_table:Creating the 'Training' table...
INFO:create_table:Training table created successfully.
INFO:create_table:Creating the 'Prediction' table...
INFO:create_table:Prediction table created successfully.
INFO:create_index:Tables created successfully.
INFO:create_index:Creating indexes on all tables in a single transaction...
INFO:create_index:Creating index on 'ScrapeTime'...
INFO:create_index:Indexes created successfully.
INFO:create_index:Creating index on 'DataPath'...
INFO:create_index:Indexes created suc

In [2]:
execute_insert("Week2Demo", "Collection", "intraQuarter", "testOutputHash")

INFO:insert_document:Inserting some documents in the Collection table...
INFO:insert_document:Documents inserted successfully!
